## Scrape ratings data

In [2]:
import pandas as pd

import time
import pickle
import random
from datetime import datetime
import re

from pymongo import MongoClient
from pprint import pprint

import requests
from bs4 import BeautifulSoup

### List of all recipes in search

In [24]:
base_url = 'https://cooking.nytimes.com/search'
recipe_urls = []
image_urls = []

In [25]:
# test
url = base_url
response = requests.get(url)
print(response.status_code)

200


In [26]:
# recipe urls
def get_recipe_and_image_urls(recipe_urls, image_urls, url):
    
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    for article in soup.find_all('article',attrs={"class":"card recipe-card"}):
        recipe_urls.append('https://cooking.nytimes.com' + article['data-url'])
        if '.jpg' in article['data-seo-image-url']:
            image_urls.append(article['data-seo-image-url'])
        else:
            image_urls.append(None)
    return recipe_urls, image_urls

In [27]:
# get the base case
recipes, images = get_recipe_and_image_urls(recipe_urls, image_urls, base_url)

In [28]:
for num in range(1,409): #409
    time.sleep(4)
    search_url = base_url + '?q=&page=' + str(num)
    recipe_urls, image_urls = get_recipe_and_image_urls(recipe_urls, image_urls, search_url)

recipes = list(zip(recipe_urls,image_urls)

In [37]:
# pickle the list of urls
with open('urls_recipe_image.pickle', 'wb') as write_file:
    pickle.dump(recipes), write_file)

In [9]:
# retrieve the list
with open('urls_recipe_image.pickle','rb') as read_file:
    recipes = pickle.load(read_file)

 ### Get recipe blurbs and article urls

In [107]:
# don't make these nulls, just don't include them
recipe_dict_list = []

for recipe in recipes[15156:]:
    
    time.sleep(.5+2*random.random())
    response = requests.get(url)
    
    if response.status_code == 200:
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        recipe_dict={
            'recipe_url':recipe[0], #url
            'image_url':recipe[1] #image url
#             'article_text':None,
#             'article_title':None,
#             'article_author':None,
#             'article_date':None
        }
        
        try:
            recipe_dict['article_url'] = assign_recipe_dict(soup.find('p',attrs={"class":"related-article"}).find('a')['href'])
        except:
            recipe_dict['article_url'] = None
        
        try:
            recipe_dict['recipe_title'] = soup.find('div',attrs={"class":"title-container"}).find('h1')['data-name']
        except:
            recipe_dict['recipe_title'] = None
        
        try:
            recipe_dict['recipe_description'] = soup.find(itemprop='description').find('p').text
        except:
            recipe_dict['recipe_description'] = None
        
        try:
            recipe_dict['recipe_author'] = soup.find('div',attrs={"class":"recipe-subhead"}).find('h3').find('a')['data-author']
        except:
            recipe_dict['recipe_author'] = None
        
    # add a dictionary with only the recipe url and the image url if the link doesn't work, and print the recipe url
    else:
        recipe_dict = {
            'recipe_url':recipe[0], #url
            'image_url':recipe[1], #image url
            'recipe_title':None,
            'recipe_description':None,
            'recipe_author':None,
            'article_url':None
#             'article_text':None,
#             'article_title':None,
#             'article_author':None,
#             'article_date':None
        }
        print(url)
    
    recipe_dict_list.append(recipe_dict)
    
    if len(recipe_dict_list)%200==0:
        with open('nyt_dicts_list.pickle', 'wb') as write_file:
            pickle.dump(recipe_dict_list, write_file)

https://cooking.nytimes.com/recipes/2461-oatmeal-raisin-cookies


In [122]:
article_list = []
counter=0
for recipe in recipe_dict_list:
    if recipe['article_url'] is not None:
        article_list.append(recipe['article_url'])
        counter+=1
print(counter)
len(article_list)
len(set(article_list))

18719


8110

In [166]:
# make list of recipes that have either a description or a url for an article
# todo- make a set of article urls since some recipes link to the same article

recipes_with_articleurl = []
recipes_with_description = []

for recipe in recipe_dict_list:
    if recipe['article_url']:
        recipes_with_articleurl.append(recipe)
    if recipe['recipe_description']:
        recipes_with_description.append(recipe)        

In [162]:
#remove these dictionaries and don't append any that don't have a recipe website (change the secondary append)
no_website_list=['https://cooking.nytimes.com/recipes/8478-candied-squash-and-ginger-relish',
'https://cooking.nytimes.com/recipes/8476-roasted-squash-puree',
'https://cooking.nytimes.com/recipes/8321-monterey-county-jail-oatmeal',
'https://cooking.nytimes.com/recipes/10625-oatmeal-cookies',
'https://cooking.nytimes.com/recipes/1372-hummus',
'https://cooking.nytimes.com/recipes/2461-oatmeal-raisin-cookies'
]
len(no_website_list)

6

In [180]:
# pickle the list
with open('nyt_dicts_list.pickle', 'wb') as write_file:
    pickle.dump(recipe_dict_list, write_file)

In [181]:
# retrieve the list
with open('nyt_dicts_list.pickle','rb') as read_file:
    recipe_dict_list = pickle.load(read_file)

### Article
#### Make a separate collection and drop in the dictionaries as you go

In [303]:
article_list = []

for doc in list(descr_col.find({'article_url': {'$ne':None}},{ '_id': 0, 'article_url':1})):
    article_list.append(doc['article_url'])

print(f"number of linked articles: {len(article_list)}")
print(f"number of unique linked articles: {len(set(article_list))}")

number of linked articles: 18719
number of unique linked articles: 8110


In [269]:
# get the scraping code to work on an example

url = 'https://www.nytimes.com/2019/05/17/dining/top-chef-eric-adjepong-west-african-food.html' #recipe_dict_list[0]['article_url']
print(url)
response = requests.get(url)
response.status_code

https://www.nytimes.com/2019/05/17/dining/top-chef-eric-adjepong-west-african-food.html


200

In [270]:
page = response.text
soup = BeautifulSoup(page, "lxml")

In [271]:
# article text

# skipping the last paragraph, it's just links, 
# strip the "(View this recipe in NYT Cooking)" text

article_body=''

for div in soup.find_all('div',attrs={"class":"StoryBodyCompanionColumn"})[:-1]:
    for next_div in div.find_all('div'):
        if next_div.find('p'):
            for p in next_div.find_all('p'):
                article_body+=p.text

print(article_body)

Nearly a year ago, the chef Eric Adjepong stood in Macau on the set of the “Top Chef,” having made it to the finals of the cooking competition show. The only thing standing between him and the title was a four-course meal, served in two parts. He’d decided to use his menu to show the judges how influential Africa’s culinary heritage is on other parts of the world, including America.“I wanted to tell the story of the trans-Atlantic slave trade from Africa through Caribbean ports, the American South and South America through food,” he said recently. “It’s an unfortunate story, but one that needs to be told.”Mr. Adjepong, 31, is first-generation Ghanaian-American, and grew up in the Little Ghana neighborhood of the Bronx. He was the first West African contestant on “Top Chef,” which just aired its 16th season, introducing some viewers and judges to dishes like fufu and egusi stew.And then, after the first course, Mr. Adjepong was eliminated, after the jerk sauce in his Jamaican-inspired s

document with article, article url, date, author, and a list of any recipes that link to it

In [281]:
# article author
# this won't work on unusual pieces, like the samin nosrat column
# also strip the by, maybe just add ot stop words
soup.find('p',attrs={"itemprop":"author creator"}).text

'By Korsha Wilson'

In [293]:
# date
# yeah I can do this but it might be easier just to strip it from the url with a regex since not all have a date
for li in soup.find_all('li'):
    if li.find('time'):
        print(li.find('time')['datetime'])

2019-05-17


In [298]:
# title
for h1 in soup.find_all('h1', attrs={'itemprop':'headline'}):
    print(h1.text)

A Chef Tells the Story of the Slave Trade Through Dinner


### Create database and collection and push the dictionaries to mongo

sudo service mongod start

In [3]:
client = MongoClient()

In [186]:
# # create db
# recipe_db = client['recipes']
# # make a collection in the db
# col = recipe_db['descriptions']
# # add the recipe description dictionaries
# inserted_descriptions = col.insert_many(recipe_dict_list)
# # check the insert
# inserted_descriptions.inserted_ids[19606]

In [4]:
rec_db = client.recipes
rec_db.list_collection_names()

['descriptions']

In [5]:
descr_col = rec_db.descriptions

In [10]:
descr_col.find_one({},{ '_id': 0})

{'recipe_url': 'https://cooking.nytimes.com/recipes/1020205-ash-reshteh-persian-greens-bean-and-noodle-soup',
 'image_url': 'https://static01.nyt.com/images/2019/05/15/dining/14Iranianrex2/merlin_154113918_721ff786-e3ef-453f-b3a0-deec1d8f8e02-threeByTwoMediumAt2X.jpg',
 'recipe_title': 'Ash Reshteh (Persian Greens, Bean and Noodle Soup)',
 'recipe_description': 'Ash reshteh’s flavor is defined by two uniquely Persian ingredients: reshteh and kashk. The soup, served during the festivities leading up to Nowruz, the Persian New Year, wouldn’t be the same without the soup noodles called reshteh, which are saltier and starchier than Italian noodles — though you could substitute linguine in a pinch. Kashk, a form of drained yogurt or whey, is saltier and more sour than Greek yogurt or sour cream. More like feta than yogurt, liquid kashk gives ash its distinct, satisfying flavor. If you can’t find liquid kashk, buy it powdered and hydrate it with warm water to the consistency of sour cream. L

In [211]:
# pull only the ones that actually have a description
counter=0
for x in descr_col.find({},{ '_id': 0})[19600:]:
    print(x)
    counter+=1
    if counter==5:
        break

{'recipe_url': 'https://cooking.nytimes.com/recipes/31-mushrooms-in-marsala-wine-funghi-alla-marsala', 'image_url': None, 'recipe_title': 'Mushrooms in Marsala Wine (Funghi Alla Marsala)', 'recipe_description': None, 'recipe_author': 'Craig Claiborne', 'article_url': 'https://www.nytimes.com/1981/02/25/garden/soupe-aux-champignons-mushroom-soup.html'}
{'recipe_url': 'https://cooking.nytimes.com/recipes/25-mushroom-soup-soupe-aux-champignons', 'image_url': None, 'recipe_title': 'Mushroom Soup (Soupe Aux Champignons)', 'recipe_description': None, 'recipe_author': 'Craig Claiborne', 'article_url': 'https://www.nytimes.com/1981/02/25/garden/soupe-aux-champignons-mushroom-soup.html'}
{'recipe_url': 'https://cooking.nytimes.com/recipes/26-meat-broth', 'image_url': None, 'recipe_title': 'Meat Broth', 'recipe_description': None, 'recipe_author': 'Craig Claiborne', 'article_url': 'https://www.nytimes.com/1981/02/25/garden/soupe-aux-champignons-mushroom-soup.html'}
{'recipe_url': 'https://cookin

In [255]:
len(list(descr_col.find({'image_url': {'$ne':None}},{ '_id': 0})))

8061

In [358]:
article_urls = []

for doc in list(descr_col.find({'article_url': {'$ne':None}},{ '_id': 0, 'article_url':1})):
    article_urls.append(doc['article_url'])

print(len(article_urls))
print(len((set(article_urls))))

18719
8110


In [369]:
q_has_desc={'recipe_description': {'$ne':None}}

print(f"number of documents with a description {len(list(descr_col.find(q_has_desc,{ '_id': 0})))}")
print(f"number of documents with a unique description {len(set(doc['recipe_description'] for doc in list(descr_col.find(q_has_desc))))}")

number of documents with a description 7069
number of documents with a unique description 6949


In [373]:
q_has_image={'image_url': {'$ne':None}}

print(f"number of documents with an image {len(list(descr_col.find(q_has_image)))}")
print(f"number of documents with a unique image {len(set(doc['image_url'] for doc in list(descr_col.find(q_has_image))))}")

number of documents with an image 8061
number of documents with a unique image 7925


In [371]:
# sometimes the same image gets used for multiple recipes from the same article, so the image date is valid for both
q_has_desc_and_image={'recipe_description': {'$ne':None}, 'image_url': {'$ne':None}}
q_has_desc_and_article={'recipe_description': {'$ne':None}, 'article_url': {'$ne':None}}

q_has_article={'article_url': {'$ne':None}}
q_has_image={'image_url': {'$ne':None}}
q_has_desc_and_article_or_image={'recipe_description': {'$ne':None}, '$or': [q_has_article, q_has_image]}

print(f"number of documents with an image and a description: {len(list(descr_col.find(q_has_desc_and_image)))}")
print(f"number of documents with a unique description and an image: {len(set(doc['recipe_description'] for doc in list(descr_col.find(q_has_desc_and_image))))}")
print('')
print(f"number of documents with an article url and a description: {len(list(descr_col.find(q_has_desc_and_article)))}")
print(f"number of documents with a unique description and an article url: {len(set(doc['recipe_description'] for doc in list(descr_col.find(q_has_desc_and_article))))}")
print('')
print(f"number of documents with a unique description and an article url or an image: {len(set(doc['recipe_description'] for doc in list(descr_col.find(q_has_desc_and_article_or_image))))}")

number of documents with an image and a description: 6333
number of documents with a unique description and an image: 6224

number of documents with an article url and a description: 6335
number of documents with a unique description and an article url: 6261

number of documents with a unique description and an article url or an image: 6934


In [329]:
print(f"repeated descriptions: {7069-6949}")
print(f"repeated images: {8061-7925}")
print(f"repeated descriptions among recipes with images: {6333-6224}")

repeated descriptions: 120
repeated images: 136
repeated descriptions among recipes with images: 109


In [385]:
# select only those docs that have an article url or an image
# if an article url exists, use its date, if no article use the image url
# update the docs to include a date
for doc in descr_col.find({})[:3]:
    print(doc['recipe_description'], '\n')


Ash reshteh’s flavor is defined by two uniquely Persian ingredients: reshteh and kashk. The soup, served during the festivities leading up to Nowruz, the Persian New Year, wouldn’t be the same without the soup noodles called reshteh, which are saltier and starchier than Italian noodles — though you could substitute linguine in a pinch. Kashk, a form of drained yogurt or whey, is saltier and more sour than Greek yogurt or sour cream. More like feta than yogurt, liquid kashk gives ash its distinct, satisfying flavor. If you can’t find liquid kashk, buy it powdered and hydrate it with warm water to the consistency of sour cream. Look for both items at a Middle Eastern grocery. 

Named for the city in southwestern Iran, salad-e Shirazi is found on practically every Iranian table at practically every meal. Think of it as an herbier, juicier, more acidic version of Greek salad, Israeli salad or Indian kachumber. It should be bright, crunchy and tart, a nice counterpoint to rich, buttery rice

In [480]:
# add the date the correspoding article was published, or the recipe image was posted, if there is one

date_regex='((?:19|20)\d\d)\/(0?[1-9]|1[012])\/([12][0-9]|3[01]|0?[1-9])'

q_has_article={'article_url': {'$ne':None}}
q_has_image={'image_url': {'$ne':None}}
q_has_desc_and_article_or_image={'recipe_description': {'$ne':None}, '$or': [q_has_article, q_has_image]}

for doc in descr_col.find(q_has_desc_and_article_or_image):
#     print(datetime.strptime(re.search(date_regex, doc['article_url']).group(), "%Y/%m/%d").date())
    try:
        desc_date = datetime.strptime(re.search(date_regex, doc['article_url']).group(), "%Y/%m/%d")
    except:
        try:
            desc_date = datetime.strptime(re.search(date_regex, doc['image_url']).group(), "%Y/%m/%d")
        except:
            continue
    descr_col.update_one({"_id": doc["_id"]}, {"$set": {"pub_date": desc_date}})

In [483]:
# looks like it works

for doc in descr_col.find(q_has_desc_and_article_or_image)[:10]:
    print(doc['pub_date'], doc['recipe_description'], '\n')

2019-05-14 00:00:00 Ash reshteh’s flavor is defined by two uniquely Persian ingredients: reshteh and kashk. The soup, served during the festivities leading up to Nowruz, the Persian New Year, wouldn’t be the same without the soup noodles called reshteh, which are saltier and starchier than Italian noodles — though you could substitute linguine in a pinch. Kashk, a form of drained yogurt or whey, is saltier and more sour than Greek yogurt or sour cream. More like feta than yogurt, liquid kashk gives ash its distinct, satisfying flavor. If you can’t find liquid kashk, buy it powdered and hydrate it with warm water to the consistency of sour cream. Look for both items at a Middle Eastern grocery. 

2019-05-14 00:00:00 Named for the city in southwestern Iran, salad-e Shirazi is found on practically every Iranian table at practically every meal. Think of it as an herbier, juicier, more acidic version of Greek salad, Israeli salad or Indian kachumber. It should be bright, crunchy and tart, a